In [2]:
import numpy as np
import numpy.linalg as lin
import plotly.graph_objects as go
import scipy.constants as syc
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [3]:
plt.style.use('seaborn-darkgrid')

font_settings = {
    'font' : 15,
    'title_font' : 18
}

def set_plot_layout(font_settings):
    plt.rcParams.update({
        'font.size': 15,
        'axes.titlesize': 18,
        'axes.labelsize': 15,
        'xtick.labelsize': 15,
        'ytick.labelsize': 15,
        'xtick.major.size': 8,
        'ytick.major.size': 8,
        'legend.fontsize': 15,
        'axes.grid': True,
        # Add more settings as needed
    })

%matplotlib qt5

In [4]:
# constants
light = syc.c
hbar = syc.hbar
fine_structure = syc.alpha
m_e = syc.physical_constants['electron mass in u'][0]
bohr = syc.physical_constants['Bohr radius'][0]
Hart_m = syc.physical_constants['hartree-inverse meter relationship'][0]
Hart = syc.physical_constants['Hartree energy'][0]
Hart_time = hbar/Hart
Hart_fr = Hart/hbar
Hart_light = bohr*Hart/(hbar*fine_structure)


# Problem 1 : Finite differences of CO
We consider a harmonic oscillator where the potential is given by

$$V(x) = \frac{1}{2} kx^2$$

Resulting in a hamiltonian of 

$$H = -\frac{\hbar^2}{2m} \frac{d^2}{dx^2} + \frac{1}{2} kx^2$$ 

with m as the reduced mass and k as the harmonic "spring constant". 
Where the reduced mass is __6.857__ u and the "spring constant" is __1857__ N/m, with $\tilde{\omega_e}$ as the vibrational constant equals __2170__ cm^-1. [1] The bondleght $r_e$ of CO is __112.8__ pm. [2] 

$$m = \frac{m_1 m_2}{m_1+m_2}$$

$$k = \omega_e^2 m$$

$$ \omega_e = 2\pi \tilde{\omega}C $$

$$ \psi(x) = \psi_i + (x-x_i)\psi_i\prime + \frac{1}{2!} (x-x_i)^2\psi_i\prime\prime + \frac{1}{3!} (x-x_i)^3\psi_i\prime\prime\prime $$

$$ \psi_i\prime\prime = \frac{\psi_{i+1}-2\psi_i+\psi_{i-1}}{\Delta^2}$$


## The anharmonic oscilator 
(based on x to the power 4)

The potential of the anhormonic oscilator given by a purtabation of the potential energy this is given by:
$$\frac{k}{2} x^2 + \frac{\gamma_3}{6} x^3 + \frac{\gamma_4}{24} x^4 + ...$$ 
we can also state $G(\nu)$ where $G(\nu)= E_\nu/hc$ and $G(\nu) = \tilde{\omega} (\nu + \frac{1}{2})$ whitout purtabations and $G(\nu) = \tilde{\omega} (\nu + \frac{1}{2}) - \tilde{x_e}\tilde{\omega_e} (\nu + \frac{1}{2})^2 + ...$

In the case of the anhormonic oscilator we use the second equation.

$$\gamma_4 = k^2/\hbar\omega$$



[1] Quantum Chemisty, D.A. Mc Quarrie (2008), p. 221

[2] https://en.wikipedia.org/wiki/Carbon_monoxide#:~:text=The%20bond%20length%20between%20the,nearly%20the%20same%20molecular%20mass

[3] http://openmopac.net/manual/vibrational_frequencies.html

In [37]:
Wn = 217000/Hart_m #m^-1 in hartree, vibrarional constant of the CO bond 
Re = 112.8*10**-12 / bohr #m in borh radia , bondlength of CO bond
Mass = 6.857/m_e #u in electron masses, reduced mass of the system
k = (2*syc.pi*Hart_light*Wn**2)*Mass #N/m spring constant
k2 = k**2 *Wn*Hart_light /hbar #N/m spring constant

: 

In [36]:
print(k/k2)

1.5457327861644038e-50


## Functions

In [6]:
def grid(steps,cls):
    '''This grid is based characteristic length scale of the harmonic oscilator (cls) and the amount of steps taken.
       It was tested for a cls of 1 that a grid size of 10 cls in total is the reasonable for first 5 eigen functions. 
       However, after introducing the constants in SI units the eigenvalues and -functions were off. 
       It could be seen that the relative kinetic to potential energy of the hamiltonian was much lower 
       indicating that the grid was to large. Decreasing the grid from 10 cls to 2 cls resolved this.
    '''

    
    x = np.linspace(-cls,cls,steps)
    dx = 2*cls/(steps-1)
   
    
    return x,dx


In [7]:
def T_fdm(size,mass,dx,hbar = hbar):
    '''finite difference method giving the kinetic energy in jules'''

    d = np.diag(2*np.ones(size))
    d1 = np.diag(-1*np.ones(size-1),1)
    d2 = np.diag(-1*np.ones(size-1),-1)
            
    return hbar**2/(2*mass*dx**2)*(d + d1 + d2)

def V_harmon(k,x):
    '''Function for the potential of the harmonic ossiclator. First add the constant k and then the x values of the grid'''      
    return np.diag(1/2*k*x**2)

def V_add (k,k2,x):
    return np.diag(1/2*k*x**2 + 1/24*k2*x**4)


In [8]:
def EIG(steps, cls, mass, k , k2 = 0, hbar = hbar): 
    [x,dx] = grid(steps,cls)
    T = T_fdm(steps,mass,dx, hbar)
    V = V_add (k,k2,x)
    H = T + V
    return np.linalg.eig(H)


## Testing the functions
The functions were tested by putting all constants to 1.

In [9]:
steps = 100
x = np.linspace(-5,5,steps)
dx = 10/(steps-1)

V = np.diag(1/2*x**2)

d = np.diag(2*np.ones(steps))
d1 = np.diag(-1*np.ones(steps-1),1)
d2 = np.diag(-1*np.ones(steps-1),-1)
T = (d + d1 + d2)/(2*dx**2)

H = T + V

In [10]:
[x,dx] = grid(100,5)
T2 = T_fdm(100,1,dx,1)
V2 = V_harmon(1,x)

H2 = T2 + V2
H == H2

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

As can be seen in the previous cell both methods one 'brute force' and one via functions yielding the same hamiltonion, meaning that we can use the functions in upcoming calcultions. 

In [11]:
#This is a check as you don't want T > V with a good margin so ~ an order greater
Energy_proportion_check = np.diag(T)[0]/np.diag(V)[0] 
print(Energy_proportion_check)  

7.840800000000001


The energy proportion check is 7.84 which is close enough to one order of magnitude. Meaning that the kinetic anergy is dominant over the potential energy, but that the potential enenrgy still has enough influence on the system to get the desired pertubation in H. As can be seen in the plot below.

In [12]:
fig, axes = plt.subplots(figsize=(9, 8))
plt.plot(x,np.diag(V), 'r', label='Potential energy')
plt.plot(x,np.diag(H), 'b', label='Totale hamitonian')
plt.ylabel('Energy [J]')
plt.xlabel('Bond leght [m]')
plt.legend()

plt.figtext(0.5, -0.1, 'Red is the potential energy of the system and Blue the total energy',
            wrap=True, horizontalalignment='center', fontsize=12)

Text(0.5, -0.1, 'Red is the potential energy of the system and Blue the total energy')

In [13]:
e,f = np.linalg.eig(H) # liniarization

fig, axes = plt.subplots(figsize=(9, 8))
plt.ylabel('Amplitude')
plt.xlabel('Step')
plt.plot(f[:,:5])
plt.legend([
        'Eigenfunction 1', 
        'Eigenfunction 2',
        'Eigenfunction 3',
        'Eigenfunction 4',
        'Eigenfunction 5'
        ])

In [23]:
e2,f2 = EIG(100,5,1,1, 0,1)
#e == e2, f == f2 

(array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]),
 array([[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  T

As can be seen again that the function EIG works as expected as it follows our previous results, which is to be expected as we only combined all the previous function into 1 and applied linirization.

##  based on characteristic length scale

In this case $ a = \sqrt{\hbar/\omega m}$ or in other words $a = \sqrt{h c \tilde{\omega}/ m}$, where a is the characteristic length scale of CO.

In [ ]:
e_CO,f_CO = EIG(100,5,1,1, 0,1)